In [6]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

# Define the ticker symbols for NASDAQ and NSE indices
nasdaq_ticker = "^IXIC"
nse_ticker = "^NSEI"

# Define the start and end dates for collecting historical data
start_date = "2010-01-01"
end_date = "2023-05-20"

# Fetch the historical data for NASDAQ index using Yahoo Finance API
nasdaq_data = yf.download(nasdaq_ticker, start=start_date, end=end_date)

# Fetch the historical data for NSE index using Yahoo Finance API
nse_data = yf.download(nse_ticker, start=start_date, end=end_date)

# Calculate the Keltner Channel indicator
def calculate_keltner_channel(data, n, k):
    data['Typical Price'] = (data['High'] + data['Low'] + data['Close']) / 3
    data['MA'] = data['Typical Price'].rolling(window=n).mean()
    data['ATR'] = data['High'] - data['Low']
    data['Upper KC'] = data['MA'] + k * data['ATR']
    data['Lower KC'] = data['MA'] - k * data['ATR']
    return data

# Calculate the Bollinger Bands indicator
def calculate_bollinger_bands(data, n, std_dev):
    data['MA'] = data['Close'].rolling(window=n).mean()
    data['Std'] = data['Close'].rolling(window=n).std()
    data['Upper BB'] = data['MA'] + std_dev * data['Std']
    data['Lower BB'] = data['MA'] - std_dev * data['Std']
    return data

# Calculate the MACD indicator
def calculate_macd(data, short_ma, long_ma, signal_ma):
    data['EMA_short'] = data['Close'].ewm(span=short_ma, adjust=False).mean()
    data['EMA_long'] = data['Close'].ewm(span=long_ma, adjust=False).mean()
    data['MACD'] = data['EMA_short'] - data['EMA_long']
    data['Signal Line'] = data['MACD'].ewm(span=signal_ma, adjust=False).mean()
    return data

# Apply the optimized parameters to the other index (choose NASDAQ for parameter optimization and NSE for trading)
n_kc = 20  # Period for moving average (Keltner Channel)
k_kc = 2.0  # Keltner Channel factor
n_bb = 20  # Period for moving average (Bollinger Bands)
std_dev_bb = 2.0  # Standard deviation factor (Bollinger Bands)
short_ma_macd = 12  # Short-term moving average period (MACD)
long_ma_macd = 26  # Long-term moving average period (MACD)
signal_ma_macd = 9  # Signal line moving average period (MACD)

# Calculate indicators for NASDAQ index
nasdaq_data = calculate_keltner_channel(nasdaq_data, n_kc, k_kc)
nasdaq_data = calculate_bollinger_bands(nasdaq_data, n_bb, std_dev_bb)
nasdaq_data = calculate_macd(nasdaq_data, short_ma_macd, long_ma_macd, signal_ma_macd)

# Calculate Keltner Channel
nse_data['Keltner Middle'] = nse_data['Close'].rolling(window=20).mean()
nse_data['Keltner Upper'] = nse_data['Keltner Middle'] + 2 * nse_data['Close'].rolling(window=20).std()
nse_data['Keltner Lower'] = nse_data['Keltner Middle'] - 2 * nse_data['Close'].rolling(window=20).std()

# Calculate Bollinger Bands
nse_data['BB Middle'] = nse_data['Close'].rolling(window=20).mean()
nse_data['BB Upper'] = nse_data['BB Middle'] + 2 * nse_data['Close'].rolling(window=20).std()
nse_data['BB Lower'] = nse_data['BB Middle'] - 2 * nse_data['Close'].rolling(window=20).std()

# Generate buy and sell signals for NSE index based on the indicators
nse_data['Signal'] = 0
nse_data.loc[nse_data['Close'] > nse_data['BB Upper'], 'Signal'] = 1
nse_data.loc[nse_data['Close'] < nse_data['BB Lower'], 'Signal'] = -1

# Record the signals, their respective dates, and returns for NSE index
nse_data.loc[nse_data['Signal'] != 0, 'Signal Date'] = nse_data.loc[nse_data['Signal'] != 0].index
nse_data['Signal Date'] = nse_data['Signal Date'].shift(1)
nse_data['Returns'] = nse_data['Signal'].shift(1) * nse_data['Close'].pct_change()


# Calculate additional metrics for NSE index
nse_data['Cumulative Returns'] = (1 + nse_data['Returns']).cumprod()
nse_data['Annualized Returns'] = nse_data['Cumulative Returns'] ** (252 / len(nse_data)) - 1
nse_data['Volatility'] = nse_data['Returns'].rolling(window=252).std()
nse_data['Max Drawdown'] = (nse_data['Cumulative Returns'].cummax() - nse_data['Cumulative Returns']) / nse_data['Cumulative Returns'].cummax()

# Print the generated signals, their respective dates, and returns for NSE index
signals = nse_data.loc[nse_data['Signal'] != 0, ['Signal', 'Signal Date', 'Returns']]
print(signals)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
            Signal Signal Date   Returns
Date                                    
2010-03-02       1         NaT  0.000000
2010-03-03       1  2010-03-02  0.014172
2010-03-04       1  2010-03-03 -0.001543
2010-04-05       1         NaT  0.000000
2010-05-04      -1         NaT -0.000000
...            ...         ...       ...
2023-01-30      -1  2023-01-27 -0.002533
2023-02-28      -1         NaT -0.000000
2023-04-11       1         NaT  0.000000
2023-04-12       1  2023-04-11  0.005084
2023-05-04       1         NaT  0.000000

[301 rows x 3 columns]
